<a href="https://colab.research.google.com/github/SIDIBEMoussa/Application-de-HMM/blob/main/TP1_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Une implémentation du pseudo code :

In [6]:
import numpy as np

"""
Compte tenu des observations
"""
O = np.array([[0,1,0,2]]) #observation en séquence
T = len(O[0]) #nombre d'observations en séquence
    
"""
Initialiser les matrices
"""
A = np.array([[.7,.3],
              [.4,.6]]) 
B = np.array([[.1,.4,.5],
              [.7,.2,.1]]) 
pi = np.array([.6,.4]) 

"""
Limites globales
"""
N = np.shape(B)[0]
M = np.shape(B)[1]

In [7]:
def init_matrices(O, N):
    """
    Créer des valeurs initiales aléatoires pour les matrices 
    qui sont des lignes stochastiques O est la séquence d'états observée 
    N  est le nombre d'états cachés disponibles
    """
    T = len(O[0])
    M = len(np.unique(O))
    A = np.random.rand(N,N)
    A = A/A.sum(axis=1)[:,None]
    B = np.random.rand(N,M)
    B = B/B.sum(axis=1)[:,None]
    pi = np.random.rand(N)
    pi = pi/pi.sum()
    c = np.zeros((T))
    alpha = np.zeros((T,N))
    beta = np.zeros((T,N))
    gamma = np.zeros((T,N))
    digam = np.zeros((T,N,N))
    return A, B, pi, alpha, beta, gamma, digam, M, T, c

In [8]:

#A, B, pi, M = init_matrices(O,N)

"""
Initialiser les itérateurs
"""
maxIters = 100
iters = 0
oldLogProb = -10**100 #moins l'infini...

c = np.zeros((T))
alpha = np.zeros((T,N))
beta = np.zeros((T,N))
gamma = np.zeros((T,N))
digam = np.zeros((T,N,N))


"""
Alpha pass
"""
def apass(A,B,pi,alpha,N,T,c):
    """Calculer alpha[0,i]"""
    c[0] = 0
    for i in range(N):
        alpha[0,i] = pi[i]*B[i,O[0,0]]
        c[0] = c[0] + alpha[0,i]
        
    """Scale alpha[0,i]"""
    c[0] = 1/c[0]
    for i in range(N):
        alpha[0,i] = c[0]*alpha[0,i]
    
    """Calculer alpha[t,i]"""
    for t in range(1,T): 
        c[t] = 0
        
        for i in range(N): 
            alpha[t,i] = 0
            for j in range(N): 
                alpha[t,i] = alpha[t,i] + alpha[t-1,j]*A[j,i]
            alpha[t,i] = alpha[t,i]*B[i,O[0,t]]
            c[t] = c[t] + alpha[t,i]
            
        c[t] = 1/c[t] #Scale alpha[t,i]
        for i in range(N):
            alpha[t,i] = c[t]*alpha[t,i]
    return alpha, c

In [9]:

"""
Beta pass
"""
def bpass(A,B,pi,beta,N,T,c):
    """beta[t,i]=1 mis à l'échelle par c[t-1]"""
    for i in range(N):
        beta[T-1,i] = c[T-1]
        
    """Beta pass"""
    for t in range(T-2,-1,-1):
        for i in range(N):
            beta[t,i] = 0
            for j in range(N):
                beta[t,i] = beta[t,i] + A[i,j]*B[j,O[0,t+1]]*beta[t+1,j]
            #Scale beta[t,i]
            beta[t,i] = c[t]*beta[t,i]
            
    return beta, c

In [10]:


"""
Digamma
"""
def digamma(A,B,pi,alpha,beta,gamma,digam,N,T):
    """Pas de normalisation puisque alpha bêta déjà mis à l'échelle"""
    for t in range(T-1):
        for i in range(N):
            gamma[t,i] = 0
            for j in range(N):
                digam[t,i,j] = alpha[t,i]*A[i,j]*B[j,O[0,t+1]]*beta[t+1,j]
                gamma[t,i] = gamma[t,i] + digam[t,i,j]
    """Cas particulier pour gamma[t-1,i]"""
    for i in range(N):
        gamma[T-1,i] = alpha[T-1,i]
    
    return gamma, digam

In [11]:

"""
Ré-estimer A, B, pi
"""
def re_est(A,B,pi,gamma,digam):
    """Ré-estimer pi"""
    for i in range(N):
        pi[i] = gamma[0,i]
        
    """Ré-estimer A"""
    for i in range(N):
        for j in range(N):
            numer = 0
            denom = 0
            for t in range(T-1):
                numer = numer + digam[t,i,j]
                denom = denom + gamma[t,i]
            A[i,j] = numer/denom
            
    """Ré-estimer B"""
    for i in range(N):
        for j in range(M):
            numer = 0
            denom = 0
            for t in range(T):
                if O[0,t] == j: numer = numer + gamma[t,i]
                denom = denom + gamma[t,i]
            B[i,j] = numer/denom
    return A, B, pi

In [12]:
"""
Calculer log P(O|lambda)
"""
def logprob(c):
    return -np.sum(np.log(c))      
         
"""
Rassembler le tout
"""
def markov(O,N):
    iters = 0
    logProb = 0
    delta = 1
    A, B, pi, alpha, beta, gamma, digam, M, T, c = init_matrices(O,N)
    while iters <= maxIters and logProb >= oldLogProb and delta >= 0.000001:
        alpha, c = apass(A,B,pi,alpha,N,T,c)
        beta, c = bpass(A,B,pi,beta,N,T,c)
        gamma, digam = digamma(A,B,pi,alpha,beta,gamma,digam,N,T)
        A, B, pi = re_est(A,B,pi,gamma,digam)
        delta1 = logProb
        logProb = logprob(c)
        delta = np.absolute(delta1 - logProb)
        iters = iters + 1
    print("Interations: ", iters)
    return A,B,pi,alpha,beta,gamma,digam

In [13]:
"""
Trouver P(O|lambda)
"""
def p_obs_lambda(alpha):
    return np.sum(alpha[T-1,:])

"""
Trouver les états cachés les plus probables
"""
def p_state(gamma):
    return np.argmax(gamma,axis=1)

### Etant donné la séquence d'observation O et les dimensions N et M, trouver le modèle lambda= (A;B; pi) qui maximise la probabilité de O ?.Cela peut être considéré comme l'apprentissage du modèle pour mieux l'adapté aux données observées.

### Étant donné lambda= (A;B;pi ) et la séquence d'observation O, trouver la séquence d'états optimale pour le processus de Markov sous-jacent ?. 

In [20]:
A,B,pi,alpha,beta,gamma,digam=markov(O,N)
pi

Interations:  11


array([2.84811254e-136, 1.00000000e+000])

In [21]:
A

array([[8.66988549e-98, 1.00000000e+00],
       [1.00000000e+00, 1.80292156e-32]])

In [22]:
B

array([[4.33495362e-98, 5.00000000e-01, 5.00000000e-01],
       [1.00000000e+00, 4.27761094e-63, 1.80292156e-32]])

In [18]:
p_obs_lambda(alpha)

1.0

# La séquence maximal

In [19]:
p_state(gamma)

array([0, 0, 1, 1])